In [7]:
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join
print(pd.__version__)

0.22.0


In [8]:
def processAdvancedStatisticsFiles(datapath, file):
    df = pd.read_excel(datapath + file)
    year = int(file[15:19])
    df['Year'] = year
    
    return df

In [9]:
# Grab all files concerning free agent contract data
datapath = './data/orig_data/'
onlyfiles = [f for f in listdir(datapath) if isfile(join(datapath, f))]

# Datasets with "Advanced" in the filename
advanced_stat_files = [k for k in onlyfiles if 'Advanced' in k]

# Combine all contract datasets into one dataframe
total_df = pd.DataFrame()

for file in advanced_stat_files:
    df = processAdvancedStatisticsFiles(datapath,file)
    # dropping players who have TOT as their team; we only maintain aggregated statistics
    dupes = df[df['Tm'] == "TOT"]
    df = df.drop_duplicates(keep=False, subset=['Player','Age'])

    df = df.append(dupes)
    
    total_df = total_df.append(df)

In [10]:
total_df[total_df['Player'] == 'Malik Allen']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
2,3,Malik Allen,PF,26,TOT,36,519,14.2,0.512,0,...,0.6,0.5,1.1,0.1,NaN,-2.1,-0.2,-2.3,0,2005
2,3,Malik Allen,PF,27,CHI,54,701,11.5,0.504,0.004,...,0,1,1,0.068,NaN,-3.8,0,-3.8,-0.3,2006
4,5,Malik Allen,PF,28,CHI,60,638,10.4,0.443,0.004,...,-0.2,1.1,1,0.073,NaN,-4.8,-0.1,-4.9,-0.5,2007
7,6,Malik Allen,PF,29,TOT,73,1096,9.8,0.5,0.006,...,0.1,0.9,1,0.045,NaN,-3.5,-0.3,-3.8,-0.5,2008
10,9,Malik Allen,PF,30,MIL,49,579,7.3,0.435,0.006,...,-0.2,0.5,0.3,0.024,NaN,-4.2,-0.1,-4.3,-0.3,2009
4,5,Malik Allen,PF,31,DEN,51,456,5.9,0.431,0.052,...,-0.3,0.3,0.1,0.009,NaN,-3.7,-0.6,-4.3,-0.3,2010
9,8,Malik Allen,PF,32,ORL,18,178,3.3,0.361,0,...,-0.1,0.2,0.2,0.046,NaN,-4.8,1.3,-3.5,-0.1,2011


In [11]:
# checking set difference to see wanted features across datasets and remove duplicated features
total_df = total_df.drop(columns=['Rk', 'Pos' , 'G' , 'MP','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'Unnamed: 19', 'Unnamed: 24'])
thiscols = np.array(total_df.columns)

In [12]:
# Write out combined data to file
total_df.to_csv(path_or_buf='./data/computed/total_player_advanced_statistics.csv')